In [1]:
import os

temp_dir="../temp_data/temp_model_dir"
temp_dir_model_original = os.path.join(temp_dir, "original")
temp_dir_model_updated = os.path.join(temp_dir, "updated")

model_original_url = "s3://aegovan-data/sagemakerresults/aimed-bert-unique-1fold-loss-2021-05-13-18-00-33-913/output/model.tar.gz"
model_updated_url = "s3://aegovan-data/embeddings/bert_aimed/bert_lowestloss"





In [2]:
get_s3_bucket_name = lambda x: x.split("//")[1].split("/")[0]
get_s3_bucket_key = lambda x: "/".join( x.split("//")[1].split("/")[1:])

In [3]:
get_s3_bucket_key(model_original_url)

'sagemakerresults/aimed-bert-unique-1fold-loss-2021-05-13-18-00-33-913/output/model.tar.gz'

In [4]:
import sagemaker, shutil
sm_session = sagemaker.session.Session()

In [5]:
sm_session.download_data(temp_dir_model_original, 
                         get_s3_bucket_name(model_original_url), 
                         get_s3_bucket_key(model_original_url) )

In [6]:
import tarfile

with tarfile.open(os.path.join(temp_dir_model_original, "model.tar.gz"), "r:gz") as so:
    so.extractall(path=temp_dir_model_original)

In [7]:
from transformers import BertForSequenceClassification

In [8]:
import torch.nn as nn 

num_labels = 7 

bert_model = BertForSequenceClassification.from_pretrained(temp_dir_model_original)
bert_model.classifier = nn.Linear(bert_model.classifier.in_features, 7)

bert_model.save_pretrained(temp_dir_model_updated)



In [9]:
sm_session.upload_data(temp_dir_model_updated, 
                         get_s3_bucket_name(model_updated_url), 
                         get_s3_bucket_key(model_updated_url) )

's3://aegovan-data/embeddings/bert_aimed/bert_lowestloss'

In [10]:
shutil.rmtree(temp_dir)